In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from src.augmentations import CustomAugmentations

import numpy as np
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

import os
os.environ['PYTHONHASHSEED'] = '42'


keras.utils.set_random_seed(42)
# tf.config.experimental.enable_op_determinism()


In [7]:
height,width=32,32
batch_size=64

In [8]:
from src.augmentations.CustomAugmentations import CustomAugmentationsTF

# Define the augmentation probabilities
p_dict = {
  "flip": 0.5,
  # "transpose": 0.5,
  "gauss_noise": 0.5,
  # "brightness_contrast": 0.5,
  # "hue_saturation_value": 0.5,
}

p_dicts = []
# for aug_type in p_dict.keys():
#     p_d = {aug_type: .5}
#     p_dicts.append(p_d)

# p_dict = {
#   "flip": 0.5,
#   "transpose": 0.5,
#   "gauss_noise": 0.5,
#   "brightness_contrast": 0.5,
#   "hue_saturation_value": 0.5 ,
# }
p_dicts.append(p_dict)

p_dicts


[{'flip': 0.5, 'gauss_noise': 0.5}]

In [9]:
import matplotlib.pyplot as plt

augmented = original_dataset.map(lambda image, label: (custom_aug.augment(image), label))



In [10]:
from copy import deepcopy
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm.notebook import tqdm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from src.kerasCNN import *
import pickle


EPOCHS = 20

histories = list()
accuracies = list()
class_accuracies = list()
conf_mats = list()

for augs in tqdm(p_dicts, desc="Augmentation"):
    aug_str = "_".join([key + str(value) for key, value in augs.items()])
    print(f"Augmentations: {aug_str}")

    for iteration in tqdm(range(5), desc="Iteration"):
        train_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/train",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="training",
        )

        val_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/valid",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=True,
            seed=iteration,
            validation_split=0.05,
            subset="validation",
        )

        test_set = tf.keras.preprocessing.image_dataset_from_directory(
            "../data/CINIC10/test",
            labels="inferred",
            label_mode="int",
            class_names=None,
            color_mode="rgb",
            batch_size=batch_size,
            image_size=(height, width),
            shuffle=False,
        )
        dnn_model = Sequential()
        imported_model = tf.keras.applications.EfficientNetV2L(
            include_top=False,
            input_shape=(height, width, 3),
            pooling="max",
            classes=10,
            weights="imagenet",
        )
        dnn_model.add(imported_model)
        dnn_model.add(Flatten())
        dnn_model.add(Dense(2048, activation="relu"))
        dnn_model.add(Dense(1024, activation="relu"))
        dnn_model.add(Dense(10, activation="softmax"))

        dnn_model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"],
    )
       

        history = {
            "loss": list(),
            "accuracy": list(),
            "val_loss": list(),
            "val_accuracy": list(),
        }

        print(f"Fitting model {iteration}")
        for epoch in tqdm(range(EPOCHS), desc="Epoch"):
            print(f"Epoch {epoch + 1}/{EPOCHS}")
            custom_aug = CustomAugmentationsTF(augs)
            train_set_aug = train_set.map(lambda x, y: (custom_aug.augment(x), y))

            hist_ = dnn_model.fit(train_set_aug, validation_data=val_set, epochs=1)
            history["loss"].append(hist_.history["loss"])
            history["accuracy"].append(hist_.history["accuracy"])
            history["val_loss"].append(hist_.history["val_loss"])
            history["val_accuracy"].append(hist_.history["val_accuracy"])

        print(f"Predicting model {iteration}")
        preds = dnn_model.predict(test_set, verbose=2)
        preds = preds.argmax(axis=1)
        classes = test_set.class_names
        test_labels = list()
        for images, labels in test_set:
            class_labels = [int(label) for label in labels]
            test_labels.extend(class_labels)
        test_labels = np.array(test_labels)

        conf_mat = confusion_matrix(test_labels, preds)
        accuracy = accuracy_score(test_labels, preds)
        print(f"Accuracy: {accuracy} for model {iteration}")
        class_accuracy = conf_mat.diagonal() / conf_mat.sum(axis=1)

        print(f"Saving model {iteration}")
        dnn_model.save(f"../models/EfficientNet_aug_{aug_str}_{iteration}.h5")
        histories.append(deepcopy(history))
        accuracies.append(deepcopy(accuracy))
        class_accuracies.append(deepcopy(class_accuracy))
        conf_mats.append(deepcopy(conf_mat))


    with open(f"results/HISTORY_EfficientNet_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(histories, f)

    with open(f"results/ACCURACY_EfficientNet_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(accuracies, f)

    with open(f"results/CLASS_ACCURACY_EfficientNet_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(class_accuracies, f)

    with open(f"results/CONF_MAT_EfficientNet_aug_{aug_str}.pkl", "wb") as f:
        pickle.dump(conf_mats, f)

Augmentation:   0%|          | 0/1 [00:00<?, ?it/s]

Augmentations: flip0.5_gauss_noise0.5


Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

Found 90000 files belonging to 10 classes.
Using 85500 files for training.
Found 90000 files belonging to 10 classes.
Using 4500 files for validation.
Found 90000 files belonging to 10 classes.
Fitting model 0


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

...

Epoch 1/20
1336/1336 [==============================] - 239s 156ms/step - loss: 1.2149 - accuracy: 0.5708 - val_loss: 0.9587 - val_accuracy: 0.6676
Epoch 2/20
1336/1336 [==============================] - 214s 160ms/step - loss: 1.0821 - accuracy: 0.6210 - val_loss: 1.1653 - val_accuracy: 0.6020
Epoch 3/20
1336/1336 [==============================] - 211s 158ms/step - loss: 1.0102 - accuracy: 0.6480 - val_loss: 1.0233 - val_accuracy: 0.6380
Epoch 4/20
1336/1336 [==============================] - 205s 153ms/step - loss: 0.9437 - accuracy: 0.6716 - val_loss: 0.8257 - val_accuracy: 0.7044
Epoch 5/20
1336/1336 [==============================] - 203s 152ms/step - loss: 0.7567 - accuracy: 0.7389 - val_loss: 0.7948 - val_accuracy: 0.7249
Epoch 6/20
1336/1336 [==============================] - 203s 152ms/step - loss: 0.8498 - accuracy: 0.7040 - val_loss: 0.9739 - val_accuracy: 0.6540
Epoch 7/20
1336/1336 [==============================] - 203s 152ms/step - loss: 0.7966 - accuracy: 0.7231 - val_

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/20
1336/1336 [==============================] - 237s 155ms/step - loss: 1.2698 - accuracy: 0.5474 - val_loss: 1.0642 - val_accuracy: 0.6267
Epoch 2/20
1336/1336 [==============================] - 203s 151ms/step - loss: 0.9425 - accuracy: 0.6725 - val_loss: 1.0421 - val_accuracy: 0.6260
Epoch 3/20
1336/1336 [==============================] - 201s 150ms/step - loss: 1.0028 - accuracy: 0.6495 - val_loss: 0.8861 - val_accuracy: 0.6862
Epoch 4/20
1336/1336 [==============================] - 202s 151ms/step - loss: 0.8050 - accuracy: 0.7219 - val_loss: 0.8947 - val_accuracy: 0.6778
Epoch 5/20
1336/1336 [==============================] - 202s 151ms/step - loss: 0.7294 - accuracy: 0.7471 - val_loss: 0.8095 - val_accuracy: 0.7216
Epoch 6/20
1336/1336 [==============================] - 202s 151ms/step - loss: 0.7034 - accuracy: 0.7589 - val_loss: 0.8091 - val_accuracy: 0.7198
Epoch 7/20
1336/1336 [==============================] - 203s 152ms/step - loss: 0.7585 - accuracy: 0.7378 - val_

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1/20
1336/1336 [==============================] - 241s 157ms/step - loss: 1.2585 - accuracy: 0.5508 - val_loss: 1.0167 - val_accuracy: 0.6567
Epoch 2/20
1336/1336 [==============================] - 203s 152ms/step - loss: 1.1519 - accuracy: 0.5967 - val_loss: 0.9487 - val_accuracy: 0.6624
Epoch 3/20
1336/1336 [==============================] - 203s 152ms/step - loss: 1.0652 - accuracy: 0.6283 - val_loss: 0.8909 - val_accuracy: 0.6916
Epoch 4/20
1336/1336 [==============================] - 203s 152ms/step - loss: 0.9887 - accuracy: 0.6567 - val_loss: 1.1866 - val_accuracy: 0.6089
Epoch 5/20
1336/1336 [==============================] - 203s 152ms/step - loss: 1.0209 - accuracy: 0.6426 - val_loss: 0.9165 - val_accuracy: 0.6829
Epoch 6/20
1336/1336 [==============================] - 204s 153ms/step - loss: 0.8856 - accuracy: 0.6911 - val_loss: 1.1293 - val_accuracy: 0.6596
Epoch 7/20
1336/1336 [==============================] - 203s 152ms/step - loss: 1.0235 - accuracy: 0.6410 - val_